Source:

huggingface: https://huggingface.co/m3hrdadfi/albert-fa-base-v2-clf-persiannews?text=%D8%AE%D9%88%D8%A8+%D9%86%D8%A8%D9%88%D8%AF+%D8%A7%D8%B5%D9%84%D8%A7#

github: https://github.com/m3hrdadfi/albert-persian

Tutoraial:https://towardsdatascience.com/fine-tuning-hugging-face-model-with-custom-dataset-82b8092f5333

In [146]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [147]:
import os
os.chdir('/content/drive/MyDrive/sharif/FineTuning/ipython(guide)')

In [148]:
!pip install transformers

In [149]:
!pip3 install sentencepiece

In [150]:
!git clone https://huggingface.co/m3hrdadfi/albert-fa-base-v2-clf-persiannews
GIT_LFS_SKIP_SMUDGE=1

fatal: destination path 'albert-fa-base-v2-clf-persiannews' already exists and is not an empty directory.


In [151]:
import tensorflow as tf
import numpy
import pandas
import re
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification,AutoConfig,TFAutoModel

In [152]:
# tokenizer = AutoTokenizer.from_pretrained("m3hrdadfi/albert-fa-base-v2")
# model = TFAutoModel.from_pretrained("m3hrdadfi/albert-fa-base-v2")

Tokenizer example:

In [153]:
text = "ما در هوشواره معتقدیم با انتقال صحیح دانش و آگاهی، همه افراد می‌توانند از ابزارهای هوشمند استفاده کنند. شعار ما هوش مصنوعی برای همه است."
tokenizer.tokenize(text)

['▁ما',
 '▁در',
 '▁هوش',
 'واره',
 '▁معتقد',
 'یم',
 '▁با',
 '▁انتقال',
 '▁صحیح',
 '▁دانش',
 '▁و',
 '▁اگاه',
 'ی',
 '،',
 '▁همه',
 '▁افراد',
 '▁می',
 '▁توانند',
 '▁از',
 '▁ابزارهای',
 '▁هوشمند',
 '▁استفاده',
 '▁کنند',
 '.',
 '▁شعار',
 '▁ما',
 '▁هوش',
 '▁مصنوعی',
 '▁برای',
 '▁همه',
 '▁است',
 '.']

In [160]:
limit_number = 750
data = pandas.read_csv('../Data/limited_to_'+str(limit_number)+'.csv',index_col=0)
data = data.dropna().reset_index(drop=True)
X = data["body"].values.tolist()
y = pandas.read_csv('../Data/limited_to_'+str(limit_number)+'.csv')
labels = []
tag=[]
for item in y['tag']:
  labels += [i for i in re.sub('\"|\[|\]|\'| |=','',item.lower()).split(",") if i!='' and i!=' ']
  tag.append([i for i in re.sub('\"|\[|\]|\'| |=','',item.lower()).split(",") if i!='' and i!=' '])
labels = list(set(labels))
mlb = MultiLabelBinarizer()
Y=mlb.fit_transform(tag)

In [155]:
##Convert tokens to string

In [164]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25)
print('train: ', len(X_train) , '\ntest: ', len(X_test) , '\nval: ', len(X_val) ,"\ny_tain:",len(y_train) )

train:  12896 
test:  4299 
val:  4299 
y_tain: 12896


In [165]:
###we would load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("m3hrdadfi/albert-fa-base-v2-clf-persiannews")

In [166]:
##Tokenize training and validation sentences:
train_encodings = tokenizer(X_train,
                            truncation=True,
                            padding=True)
val_encodings = tokenizer(X_val,
                            truncation=True,
                            padding=True)
test_encodings=tokenizer(X_test,
                            truncation=True,
                            padding=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [168]:
##Here we map sentences with labels, there is no need to pass label into fit function separately
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    y_val
))

In [193]:
###We need to get a pre-trained Hugging Face model
# model = AutoModelForSequenceClassification.from_pretrained("m3hrdadfi/albert-fa-base-v2-clf-persiannews")
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=78)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_projector', 'vocab_layer_norm', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_107', 'pre_classifier', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use 

In [194]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])


In [195]:
y_train.shape

(12896, 78)

In [197]:
##Fine-tuning
model.fit(train_dataset.shuffle(100),
          epochs=3,
          batch_size=16,
          validation_data=val_dataset.shuffle(100))

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


Epoch 1/3


ValueError: ignored

In [ ]:
model.save_pretrained("/tmp/sentiment_custom_model")

In [ ]:
loaded_model = TFDistilBertForSequenceClassification.from_pretrained("/tmp/sentiment_custom_model")

In [ ]:
predict_input = tokenizer.encode(X_test,
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")tf_output = loaded_model.predict(predict_input)[0]

In [ ]:
tf_prediction = tf.nn.softmax(tf_output, axis=1).numpy()[0]